In [1]:
"""
    Código de python para obtener los datos de MeLi, específicamente de MLA(Argentina),
    los datos obtenidos son cargados en un dataframe y después cargados en un archivo csv para su posterior análisis.
    1)Se realiza una petición a todas las categorías y se guardan las categorías en un csv.
    2)Por cada categoría se realiza una petición a la API para obtener los primeros 50 registros,
    se incrementa de 50 en 50 hasta llegar a 1000 para obtener los primeros 1000 registros de cada categoría 
"""
import pandas as pd
import requests
import time

#Se consulta la api
category_request = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
categories = category_request.json()
category_list = []
for category in categories:
    category_list.append((category['id'],category['name']))

    


In [2]:
#Se crea el dataframe de categorías
df_categories = pd.DataFrame(data=category_list,columns=['id','name'])

In [9]:
#Se descarga en un csv para su posterior análisis.
df_categories.to_csv("categories.csv", index=False)

In [6]:
#Se itera sobre cada categoría para hacer la petición al API y se incrementa de 50 en 50
#Se hace una descomposición del JSON que devuelve la API, se obtienen únicamente los campos
#que consideré importantes para poder hacer un análisis que nos permita crear una clasificación
#de los sellers que son de mejor interés para MeLi y son cargados en un dataframe para su posterior análisis
results_list = []

session = requests.Session()

for category_param in category_list:

    offset_param = 0
    category_param_id = category_param[0]
    try:
        while(offset_param<=1000):
            url = 'https://api.mercadolibre.com/sites/MLA/search?category={0}&offset={1}'.format(category_param_id,offset_param)
            cats = session.get(url)
            if cats.status_code == 200:
                cats_json = cats.json()
                offset_param += cats_json['paging']['limit']
                results = cats_json['results']

                for item in results:
                    id = item['id']
                    title = item['title']
                    condition = item['condition']
                    permalink = item['permalink']
                    accepts_mercadopago = item['accepts_mercadopago']
                    listing_type_id = item['listing_type_id']
                    site_id = item['site_id']
                    original_category = category_param[0]
                    category_id = item['category_id']
                    catalog_product_id = item['catalog_product_id']
                    currency_id  = item['price']
                    price = item['price']
                    original_price = item['original_price']
                    installments_quantity = item['installments']['quantity'] if item['installments'] is not None else ''
                    installments_amount = item['installments']['amount'] if item['installments'] is not None else ''
                    installments_rate = item['installments']['rate'] if item['installments'] is not None else ''
                    seller_id = item['seller']['id'] if 'seller' in item and item['seller'] is not None else ''
                    seller_nickname = item['seller']['nickname']
                    official_store_id = item['official_store_id']
                    official_store_name = item['official_store_name'] if 'official_store_name' in item else ''
                    available_quantity = item['available_quantity']
                    use_thumbnail_id = item['use_thumbnail_id']
                    item_info = (id,title,condition,permalink,accepts_mercadopago,listing_type_id,site_id,original_category,
                                category_id,catalog_product_id,currency_id,price,original_price,installments_quantity,installments_amount,
                                installments_rate,seller_id,seller_nickname,official_store_id,
                                official_store_name,available_quantity,use_thumbnail_id)
                    results_list.append(item_info)
            else:
                print(f"Error en la solicitud: {cats.status_code}")
            time.sleep(1)
    finally:
        session.close()            
df_items = pd.DataFrame(data=results_list,
                        columns=['id','title','condition','permalink','accepts_mercadopago','listing_type_id','site_id','original_category',
                                 'category_id','catalog_product_id','currency_id','price','original_price','installments_quantity','installments_amount',
                                 'installments_rate','seller_id','seller_nickname','official_store_id',
                                 'official_store_name','available_quantity','use_thumbnail_id'])


In [7]:
df_items 

,id,title,condition,permalink,accepts_mercadopago,listing_type_id,site_id,original_category,category_id,catalog_product_id,...,original_price,installments_quantity,installments_amount,installments_rate,seller_id,seller_nickname,official_store_id,official_store_name,available_quantity,use_thumbnail_id
0,MLA1583153780,Microfibra 40x80 Cm Autos Detailing Secado Rap...,new,https://www.mercadolibre.com.ar/microfibra-40x...,True,gold_special,MLA,MLA5725,MLA424491,MLA19601255,...,NaN,,,,290403581,LS PALERMO,NaN,,500,True
1,MLA1277641650,Pack 4 Paños Microfibra Multiuso Limpieza Auto...,new,https://www.mercadolibre.com.ar/pack-4-panos-m...,True,gold_special,MLA,MLA5725,MLA424491,MLA20023033,...,NaN,,,,702150429,GLOBALNAVET,NaN,,150,True
2,MLA1378229931,Casco Moto Vertigo V50 Dark Edicion Especial. ...,new,https://articulo.mercadolibre.com.ar/MLA-13782...,True,gold_special,MLA,MLA5725,MLA4712,MLA25745788,...,98535.00,6,13935.98,12.74,263770607,GRAVEDADX,NaN,,50,False
3,MLA851109741,Carcasa Llave Navaja Peugeot 2 Bot 307 308 C/...,new,https://articulo.mercadolibre.com.ar/MLA-85110...,True,gold_special,MLA,MLA5725,MLA373345,None,...,8255.74,6,1670.05,34.86,435859328,KEYFAD,3272.0,Keyfad,50,True
4,MLA1148567142,Bateria Alpina 12n5-3b Gel Zanella Zb 110 125 ...,new,https://articulo.mercadolibre.com.ar/MLA-11485...,True,gold_special,MLA,MLA5725,MLA45502,None,...,33228.00,6,4506.41,12.74,685241555,ALPINA ARG,2541.0,Alpina,250,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33033,MLA1439582821,Sahumerios Bangladesh Simple Empaste X100 Unid...,new,https://www.mercadolibre.com.ar/sahumerios-ban...,True,gold_special,MLA,MLA1953,MLA387586,MLA37701674,...,NaN,,,,191914203,G.UTOPIA,54275.0,Grupo Utopia,150,True
33034,MLA1389640875,Raw Armador De Conos Automatico Six Shooter 6 ...,new,https://www.mercadolibre.com.ar/raw-armador-de...,True,gold_special,MLA,MLA1953,MLA417903,MLA27240563,...,NaN,6,10451.65,34.86,251681571,PLANTOLUEGOEXISTOGROWSHOP,54445.0,Planto Luego Existo,1,True
33035,MLA1457772836,Sahumerios Sagrada Madre Artesano Natural Inci...,new,https://www.mercadolibre.com.ar/sahumerios-sag...,True,gold_special,MLA,MLA1953,MLA387586,MLA24514489,...,NaN,6,1231.72,34.86,191914203,G.UTOPIA,54275.0,Grupo Utopia,1,True
33036,MLA1705231190,Pastilla Sagrada Madre 7 Dias De Limpieza Frag...,new,https://www.mercadolibre.com.ar/pastilla-sagra...,True,gold_special,MLA,MLA1953,MLA387586,MLA25144274,...,NaN,,,,789683079,ORIENTE_MISTICO,NaN,,1,True


In [8]:
#Se crea un csv con los datos para su posterior análisis
df_items.to_csv("dataset.csv",index=False)